# 1. <i>Import libraries</i>

In [1]:
from pandas import read_csv

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

from torch.autograd import Variable
from torch.nn.functional import relu
from torch import Tensor, optim, nn, max

# 2. Muat kemudian bagi <i>dataset</i>

In [2]:
data = read_csv('iris.csv')
data.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [3]:
data.loc[data.species == 'Iris-setosa', 'species'] = 0
data.loc[data.species == 'Iris-versicolor', 'species'] = 1
data.loc[data.species == 'Iris-virginica', 'species'] = 2


x_train, x_test, y_train, y_test = train_test_split(data[data.columns[0:4]].values,
                                                    data.species.values,
                                                    test_size = 0.3)

x_train = Variable(Tensor(x_train).float())
x_test = Variable(Tensor(x_test).float())
y_train = Variable(Tensor(y_train.astype(float)))
y_test = Variable(Tensor(y_test.astype(float)))

# 3. Buat jariangan saraf tiruannya

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(4, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, 3)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, X):
        X = relu(self.fc1(X))
        X = self.fc2(X)
        X = self.fc3(X)
        X = self.softmax(X)

        return X

In [5]:
Net()

Net(
  (fc1): Linear(in_features=4, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=3, bias=True)
  (softmax): Softmax(dim=1)
)

# 4. Sesuaikan parameter

In [6]:
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1)

# 5. Latih jaringan saraf tiruannya

In [7]:
for epoch in range(100):
    optimizer.zero_grad()
    
    out = net(x_train)
    loss = criterion(out, y_train.long())
    
    loss.backward()
    optimizer.step()
    
    print('Epoch ke', epoch, 'nilai loss', loss.data)

Epoch ke 0 nilai loss tensor(1.0788)
Epoch ke 1 nilai loss tensor(1.0626)
Epoch ke 2 nilai loss tensor(1.0449)
Epoch ke 3 nilai loss tensor(1.0258)
Epoch ke 4 nilai loss tensor(1.0059)
Epoch ke 5 nilai loss tensor(0.9859)
Epoch ke 6 nilai loss tensor(0.9667)
Epoch ke 7 nilai loss tensor(0.9489)
Epoch ke 8 nilai loss tensor(0.9326)
Epoch ke 9 nilai loss tensor(0.9181)
Epoch ke 10 nilai loss tensor(0.9051)
Epoch ke 11 nilai loss tensor(0.8934)
Epoch ke 12 nilai loss tensor(0.8828)
Epoch ke 13 nilai loss tensor(0.8732)
Epoch ke 14 nilai loss tensor(0.8644)
Epoch ke 15 nilai loss tensor(0.8563)
Epoch ke 16 nilai loss tensor(0.8488)
Epoch ke 17 nilai loss tensor(0.8418)
Epoch ke 18 nilai loss tensor(0.8353)
Epoch ke 19 nilai loss tensor(0.8293)
Epoch ke 20 nilai loss tensor(0.8235)
Epoch ke 21 nilai loss tensor(0.8180)
Epoch ke 22 nilai loss tensor(0.8128)
Epoch ke 23 nilai loss tensor(0.8078)
Epoch ke 24 nilai loss tensor(0.8029)
Epoch ke 25 nilai loss tensor(0.7982)
Epoch ke 26 nilai loss

# 6. Prediksi dan evaluasi modelnya

In [8]:
hasil_prediksi = net(x_test)
hasil_prediksi = max(hasil_prediksi, 1)[1]

In [9]:
confusion_matrix(y_test.data, hasil_prediksi.data)

array([[14,  0,  0],
       [ 0, 11,  5],
       [ 0,  0, 15]])

In [10]:
print(classification_report(y_test.data, hasil_prediksi.data))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        14
         1.0       1.00      0.69      0.81        16
         2.0       0.75      1.00      0.86        15

    accuracy                           0.89        45
   macro avg       0.92      0.90      0.89        45
weighted avg       0.92      0.89      0.89        45

